In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Set seed
tf.random.set_seed(42)

In [4]:
# Path of all datasets
num_datasets = ["./data/NORM_BCW.csv", "./data/NORM_Ecoli.csv", "./data/NORM_Iris.csv", "./data/NORM_ISOLET.csv", "./data/NORM_SDD.csv", "./data/NORM_PBC.csv", "./data/NORM_CMSC.csv", "./data/NORM_MagicGT.csv", "./data/NORM_Wine.csv"]
cat_datasets = ["./data/OH_BalanceScale.csv", "./data/OH_CarEvaluation.csv", "./data/OH_HayesRoth.csv", "./data/OH_Chess.csv", "./data/OH_Lenses.csv", "./data/OH_Lymphography.csv", "./data/OH_Nursery.csv", "./data/OH_SoybeanSmall.csv", "./data/OH_TicTacToe.csv"]
mix_datasets = ["./data/OH_NORM_DefaultOfCCC.csv", "./data/OH_NORM_StudentPerf.csv", "./data/OH_NORM_Adult.csv", "./data/OH_NORM_InternetAdv.csv", "./data/OH_NORM_StatlogGC.csv"]

In [5]:
def data_splitter(df, classes):
    
    # Sets
    X_trains = []
    X_tests = []
    y_trains = []
    y_tests = []
    
    # Iteration for each class
    for c in classes:
        df_s = df[df['output']==c]
        X_train, X_test, y_train, y_test = train_test_split(df_s.drop(columns=['output']), 
                                                            df_s['output'], test_size=0.1, random_state=42)
        X_trains.append(X_train)
        X_tests.append(X_test)
        y_trains.append(y_train)
        y_tests.append(y_test)
    
    return pd.concat(X_trains), pd.concat(X_tests), pd.concat(y_trains), pd.concat(y_tests)

In [6]:
def generate_models(paths_datasets):
    for path_dataset in paths_datasets:
        # Generate df
        df = pd.read_csv(path_dataset)
        
        # Get the name of the dataset
        ds_name = path_dataset.split('.')[1].split('_')[-1]
        
        # Get the possible classes of DS
        classes = list(df['output'].unique())
        
        # Split DataFrame to train and test
        X_train, X_test, y_train, y_test = data_splitter(df, classes)
        
        for c in classes:
            # The selected class is 0 and all others are 1
            y_train = y_train.copy().apply(lambda x: 0 if x==c else 1)
            y_test = y_test.copy().apply(lambda x: 0 if x==c else 1)
            proportion_0 = y_train[y_train==0.0].shape[0]/y_train.shape[0]
            proportion_1 = y_train[y_train==1.0].shape[0]/y_train.shape[0]
            
            if proportion_0 > 0 and proportion_1 > 0:
                # Save model train Data indexes
                pd.DataFrame(y_train.index).rename(columns={0: 'index'}).to_csv(f'./idxstrain/{str(int(c))}_{ds_name}.csv', index=False)

                #Save test Data indexes
                pd.DataFrame(y_test.index).rename(columns={0: 'index'}).to_csv(f'./idxstest/{str(int(c))}_{ds_name}.csv', index=False)

                # Create model
                model = keras.Sequential(
                    [layers.Dense(X_train.shape[1]*2, activation="relu", name="layer1"), # For ISOLET, InternetAdv 4,
                     layers.Dense(X_train.shape[1]*1,  activation="relu", name="layer2"), # For ISOLET, InternetAdv 2
                     layers.Dense(1, activation="sigmoid", name="outputLayer"),
                    ])

                # Configure callbacks
                my_callbacks = [
                    tf.keras.callbacks.ModelCheckpoint(filepath=f'./tempModels/{str(int(c))}_{ds_name}_model.temp.h5'),
                    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
                    tf.keras.callbacks.EarlyStopping( monitor="loss", min_delta=0.1, patience=10, mode="auto")
                ]

                # Configure optimizer
                opt = tf.keras.optimizers.RMSprop(
                    learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False, name='RMSprop')

                # Compile
                model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])

                # Train
                model.fit(X_train, y_train, epochs=100, callbacks=my_callbacks, class_weight={0.0: 1/proportion_0, 1.0: 1/proportion_1})

                #Get Accuracies
                train_acc = (pd.DataFrame(model.predict(X_train))[0].map(round)==y_train.reset_index(drop=True)).sum()/X_train.shape[0]
                test_acc = (pd.DataFrame(model.predict(X_test))[0].map(round)==y_test.reset_index(drop=True)).sum()/X_test.shape[0]

                print(f"\n\nModel for {ds_name}:\nTrain Accuracy:{train_acc}\nTest Accuracy:{test_acc}\nClass Balance={y_train.sum()/y_train.shape[0]}\n\n")

                # Save
                model.save("./models/"+str(int(c))+"_"+ds_name+".h5")

                # Release GPU memory
                tf.keras.backend.clear_session()
                del model

In [7]:
# Create models for numerical datasets
generate_models(num_datasets)
# Create models for categorical datasets
generate_models(cat_datasets)
# Create models for mixed datasets
generate_models(mix_datasets)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1/6 [====>.........................] - ETA: 0s - loss: 1.2201 - accuracy: 0.7500WARNING:tensorflow:From /home/rmazzine/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
6/6 [==============================] - 0s 7ms/step - loss: 1.4168 - accuracy: 0.5763
Epoch 2/100
6/6 [======================

6/6 [==============================] - 0s 10ms/step - loss: 1.4151 - accuracy: 0.5141
Epoch 2/100
6/6 [==============================] - 0s 4ms/step - loss: 1.2853 - accuracy: 0.6384
Epoch 3/100
6/6 [==============================] - 0s 4ms/step - loss: 1.2290 - accuracy: 0.7119
Epoch 4/100
6/6 [==============================] - 0s 4ms/step - loss: 1.1793 - accuracy: 0.7175
Epoch 5/100
6/6 [==============================] - 0s 4ms/step - loss: 1.1363 - accuracy: 0.7966
Epoch 6/100
6/6 [==============================] - 0s 3ms/step - loss: 1.1037 - accuracy: 0.7853
Epoch 7/100
6/6 [==============================] - 0s 3ms/step - loss: 1.0625 - accuracy: 0.8192
Epoch 8/100
6/6 [==============================] - 0s 3ms/step - loss: 1.0321 - accuracy: 0.7966
Epoch 9/100
6/6 [==============================] - 0s 4ms/step - loss: 1.0072 - accuracy: 0.8249
Epoch 10/100
6/6 [==============================] - 0s 3ms/step - loss: 0.9732 - accuracy: 0.8362
Epoch 11/100
6/6 [======================

10/10 [==============================] - 0s 3ms/step - loss: 0.3674 - accuracy: 0.9732
Epoch 18/100
10/10 [==============================] - 0s 3ms/step - loss: 0.3438 - accuracy: 0.9732
Epoch 19/100
10/10 [==============================] - 0s 4ms/step - loss: 0.3216 - accuracy: 0.9732
Epoch 20/100
10/10 [==============================] - 0s 3ms/step - loss: 0.3013 - accuracy: 0.9765
Epoch 21/100
10/10 [==============================] - 0s 6ms/step - loss: 0.2833 - accuracy: 0.9732
Epoch 22/100
10/10 [==============================] - 0s 3ms/step - loss: 0.2686 - accuracy: 0.9732
Epoch 23/100
10/10 [==============================] - 0s 3ms/step - loss: 0.2550 - accuracy: 0.9732
Epoch 24/100
10/10 [==============================] - 0s 3ms/step - loss: 0.2430 - accuracy: 0.9765
Epoch 25/100
10/10 [==============================] - 0s 3ms/step - loss: 0.2319 - accuracy: 0.9732
Epoch 26/100
10/10 [==============================] - 0s 3ms/step - loss: 0.2221 - accuracy: 0.9732
Epoch 27/100


5/5 [==============================] - 0s 4ms/step - loss: 0.9795 - accuracy: 0.8741
Epoch 16/100
5/5 [==============================] - 0s 4ms/step - loss: 0.9659 - accuracy: 0.8741
Epoch 17/100
5/5 [==============================] - 0s 4ms/step - loss: 0.9504 - accuracy: 0.8815
Epoch 18/100
5/5 [==============================] - 0s 4ms/step - loss: 0.9350 - accuracy: 0.8815
Epoch 19/100
5/5 [==============================] - 0s 4ms/step - loss: 0.9202 - accuracy: 0.8815
Epoch 20/100
5/5 [==============================] - 0s 4ms/step - loss: 0.9063 - accuracy: 0.8815
Epoch 21/100
5/5 [==============================] - 0s 4ms/step - loss: 0.8921 - accuracy: 0.8815
Epoch 22/100
5/5 [==============================] - 0s 4ms/step - loss: 0.8775 - accuracy: 0.8815
Epoch 23/100
5/5 [==============================] - 0s 4ms/step - loss: 0.8635 - accuracy: 0.8815
Epoch 24/100
5/5 [==============================] - 0s 4ms/step - loss: 0.8491 - accuracy: 0.8815
Epoch 25/100
5/5 [===============

5/5 [==============================] - 0s 4ms/step - loss: 0.8799 - accuracy: 0.9926
Epoch 30/100
5/5 [==============================] - 0s 5ms/step - loss: 0.8682 - accuracy: 0.9926
Epoch 31/100
5/5 [==============================] - 0s 5ms/step - loss: 0.8573 - accuracy: 0.9926
Epoch 32/100
5/5 [==============================] - 0s 5ms/step - loss: 0.8461 - accuracy: 1.0000
Epoch 33/100
5/5 [==============================] - 0s 5ms/step - loss: 0.8357 - accuracy: 1.0000
Epoch 34/100
5/5 [==============================] - 0s 5ms/step - loss: 0.8257 - accuracy: 1.0000
Epoch 35/100
5/5 [==============================] - 0s 5ms/step - loss: 0.8157 - accuracy: 1.0000
Epoch 36/100
5/5 [==============================] - 0s 4ms/step - loss: 0.8062 - accuracy: 1.0000
Epoch 37/100
5/5 [==============================] - 0s 5ms/step - loss: 0.7969 - accuracy: 1.0000
Epoch 38/100
5/5 [==============================] - 0s 4ms/step - loss: 0.7881 - accuracy: 1.0000
Epoch 39/100
5/5 [===============

...

27/27 [==============================] - 0s 2ms/step - loss: 0.1211 - accuracy: 0.9907
Epoch 22/100
27/27 [==============================] - 0s 2ms/step - loss: 0.1076 - accuracy: 0.9907
Epoch 23/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0897 - accuracy: 0.9954
Epoch 24/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0756 - accuracy: 0.9954
Epoch 25/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0637 - accuracy: 0.9988
Epoch 26/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0564 - accuracy: 0.9977
Epoch 27/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0475 - accuracy: 0.9977
Epoch 28/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0453 - accuracy: 0.9988
Epoch 29/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0367 - accuracy: 0.9988
Epoch 30/100
27/27 [==============================] - 0s 2ms/step - loss: 0.0306 - accuracy: 1.0000
Epoch 31/100


19/19 [==============================] - 0s 2ms/step - loss: 0.7809 - accuracy: 0.8422
Epoch 14/100
19/19 [==============================] - 0s 2ms/step - loss: 0.7706 - accuracy: 0.8302
Epoch 15/100
19/19 [==============================] - 0s 3ms/step - loss: 0.7427 - accuracy: 0.8456
Epoch 16/100
19/19 [==============================] - 0s 2ms/step - loss: 0.7211 - accuracy: 0.8542
Epoch 17/100
19/19 [==============================] - 0s 2ms/step - loss: 0.6977 - accuracy: 0.8559
Epoch 18/100
19/19 [==============================] - 0s 2ms/step - loss: 0.6746 - accuracy: 0.8731
Epoch 19/100
19/19 [==============================] - 0s 2ms/step - loss: 0.6530 - accuracy: 0.8868
Epoch 20/100
19/19 [==============================] - 0s 2ms/step - loss: 0.6345 - accuracy: 0.8816
Epoch 21/100
19/19 [==============================] - 0s 2ms/step - loss: 0.6163 - accuracy: 0.8902
Epoch 22/100
19/19 [==============================] - 0s 2ms/step - loss: 0.5918 - accuracy: 0.9022
Epoch 23/100


19/19 [==============================] - 0s 2ms/step - loss: 0.4734 - accuracy: 0.9125
Epoch 26/100
19/19 [==============================] - 0s 2ms/step - loss: 0.4547 - accuracy: 0.9245
Epoch 27/100
19/19 [==============================] - 0s 2ms/step - loss: 0.4300 - accuracy: 0.9348
Epoch 28/100
19/19 [==============================] - 0s 2ms/step - loss: 0.4113 - accuracy: 0.9400
Epoch 29/100
19/19 [==============================] - 0s 2ms/step - loss: 0.3892 - accuracy: 0.9400
Epoch 30/100
19/19 [==============================] - 0s 2ms/step - loss: 0.3692 - accuracy: 0.9434
Epoch 31/100
19/19 [==============================] - 0s 2ms/step - loss: 0.3424 - accuracy: 0.9520
Epoch 32/100
19/19 [==============================] - 0s 2ms/step - loss: 0.3331 - accuracy: 0.9571
Epoch 33/100
19/19 [==============================] - 0s 2ms/step - loss: 0.3166 - accuracy: 0.9554
Epoch 34/100
19/19 [==============================] - 0s 2ms/step - loss: 0.2890 - accuracy: 0.9588
Epoch 35/100


67/67 [==============================] - 1s 21ms/step - loss: 0.4087 - accuracy: 0.9326
Epoch 2/100
67/67 [==============================] - 1s 20ms/step - loss: 0.1775 - accuracy: 0.9628
Epoch 3/100
67/67 [==============================] - 1s 18ms/step - loss: 0.1307 - accuracy: 0.9821
Epoch 4/100
67/67 [==============================] - 1s 20ms/step - loss: 0.0670 - accuracy: 0.9877
Epoch 5/100
67/67 [==============================] - 1s 22ms/step - loss: 0.0834 - accuracy: 0.9896
Epoch 6/100
67/67 [==============================] - 1s 20ms/step - loss: 0.0539 - accuracy: 0.9915
Epoch 7/100
67/67 [==============================] - 1s 20ms/step - loss: 0.0455 - accuracy: 0.9934
Epoch 8/100
67/67 [==============================] - 1s 20ms/step - loss: 0.0422 - accuracy: 0.9934
Epoch 9/100
67/67 [==============================] - 2s 23ms/step - loss: 0.0398 - accuracy: 0.9953
Epoch 10/100
67/67 [==============================] - 1s 21ms/step - loss: 0.0516 - accuracy: 0.9943
Epoch 11/10

29/29 [==============================] - 0s 2ms/step - loss: 0.2379 - accuracy: 0.9600
Epoch 29/100
29/29 [==============================] - 0s 2ms/step - loss: 0.2012 - accuracy: 0.9733
Epoch 30/100
29/29 [==============================] - 0s 2ms/step - loss: 0.1841 - accuracy: 0.9778
Epoch 31/100
29/29 [==============================] - 0s 2ms/step - loss: 0.1631 - accuracy: 0.9800
Epoch 32/100
29/29 [==============================] - 0s 2ms/step - loss: 0.1515 - accuracy: 0.9844
Epoch 33/100
29/29 [==============================] - 0s 2ms/step - loss: 0.1304 - accuracy: 0.9889
Epoch 34/100
29/29 [==============================] - 0s 2ms/step - loss: 0.1180 - accuracy: 0.9867
Epoch 35/100
29/29 [==============================] - 0s 2ms/step - loss: 0.1088 - accuracy: 0.9900
Epoch 36/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0912 - accuracy: 0.9900
Epoch 37/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0794 - accuracy: 0.9956
Epoch 38/100
